## 1)EDA
##### 1.1)Importing LIbraries.
##### 1.2)Exploration and Wordclouds.
##### 1.3)Making some new features.
## 2)LB~(0.6)
##### 2.1)Countvectorizers
##### 2.1)Baseline Logistic Regression Model

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')


##### Glimpse of Data

In [ ]:
train.head()

In [ ]:
print(train.id[0],'---')
print(train.text[0])

## Is it Balanced Data?

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(train['author']);
plt.title('Countplot for Authours_');
plt.xlabel('Authors_',fontsize=20);

So the anwer is yes!

In [ ]:
df_eap = train[train.author=='EAP']
df_hpl = train[train.author=='HPL']
df_mws = train[train.author=='MWS']

## Generating the word cloud from Authors Dictionary!
1)EAP

In [ ]:
df_eap.text
dic= (' '.join(df_eap['text']))

wordcloud = WordCloud(width = 1000, height = 500,stopwords=STOPWORDS).generate(dic)

plt.figure(figsize=(15,5));
plt.imshow(wordcloud);
plt.axis('off');
plt.title('Word Cloud for EAP');

*2)HPL*

In [ ]:
dic= (' '.join(df_hpl['text']))

wordcloud = WordCloud(width = 1000, height = 500,stopwords=STOPWORDS).generate(dic)

plt.figure(figsize=(15,5));
plt.imshow(wordcloud);
plt.axis('off');
plt.title('Word Cloud for HPL');

*MWS*

In [ ]:

dic= (' '.join(df_mws['text']))

wordcloud = WordCloud(width = 1000, height = 500,stopwords=STOPWORDS).generate(dic)

plt.figure(figsize=(15,5));
plt.imshow(wordcloud);
plt.axis('off');
plt.title('Word Cloud for MWS');

### Its Enough EDA author wise, Now lets jump to train DataSet and get sum features.

**1)Lenght of Words**

In [ ]:
train["length"] = train["text"].apply(lambda x: len(str(x).split()))
test["length"] = test["text"].apply(lambda x: len(str(x).split()))
print(train['length'].head())

**2)Removing the Stopwords**

In [ ]:
train["stp_len"] = train["text"].apply(lambda x: len([i for i in str(x).lower().split() if i in stopwords.words("english")]))
test["stp_len"] = test["text"].apply(lambda x: len([i for i in str(x).lower().split() if i in stopwords.words("english")]))
print(train['stp_len'].head())

## Lets see mean length of each writter.

In [ ]:
print(train.groupby(by=['author'])['length'].mean())
train.groupby(by=['author'])['length'].mean().plot(kind='bar');

## Lets see which user has widest dictionary!

In [ ]:
print(train.groupby(by=['author'])['stp_len'].mean())
train.groupby(by=['author'])['stp_len'].mean().plot(kind='bar');

## In our analysis the punctuation marks can play a very role

In [ ]:
import string
train["punct"] =train['text'].apply(lambda x: len([i for i in str(x) if i in string.punctuation]) )
test["punct"] =test['text'].apply(lambda x: len([i for i in str(x) if i in string.punctuation]) )

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
print(train.groupby(by=['author'])['punct'].mean())
train.groupby(by=['author'])['punct'].mean().plot(kind='bar');

# Predictions on test Data-

### CountVectorizer
<p>We call <strong>vectorization</strong> the general process of turning a collection
of text documents into numerical feature vectors. This specific strategy
(tokenization, counting and normalization) is called the <strong>Bag of Words</strong>
or “Bag of n-grams” representation. Documents are described by word
occurrences while completely ignoring the relative position information
of the words in the document.</p>

In [ ]:
vect = CountVectorizer(ngram_range=(1,2),min_df=5).fit(train['text'])
train_vectorized = vect.transform(train['text'])
len(vect.get_feature_names())

### This technique called Stacking will help us to combine our extracted features with X_test

In [ ]:
k=['length', 'stp_len', 'punct']
arr = np.array(train[k])
import scipy
stack = train[['length', 'stp_len', 'punct']]
train_vectorized = scipy.sparse.hstack([train_vectorized, stack])

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=.03,n_jobs=-1)
model.fit(train_vectorized, train['author'])

### This technique called Stacking will help us to combine our extracted features with X_test

In [ ]:
k=np.array(test[['length', 'stp_len', 'punct']])
X_test = vect.transform(test['text'] )
X_test = scipy.sparse.hstack([X_test, k])

## Making Predictions on test set

In [ ]:
df = pd.DataFrame(model.predict_proba(X_test),index=test['id'],columns=['EAP','HPL','MWS'])
df.head()

1. ## Saving as Submission file LB ~0.6

In [ ]:
df.to_csv('Submission1.csv')

#### This was baseline score.
#### I will keep on updating it.